1.이미지 얼굴 검출하기

In [1]:
import numpy as np
import cv2

font = cv2.FONT_HERSHEY_SIMPLEX
def faceDetect(frame):
    eye_detect = True
    face_cascade = cv2.CascadeClassifier('datasets/haarcascade_frontface.xml')
    eye_cascade = cv2.CascadeClassifier('datasets/haarcascade_eye.xml')
    info = ''
    
            
    if eye_detect:
        info = 'Eye Detection On'
    else:
        info = 'Eye Detection Off'
            
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    cv2.putText(frame, info, (5, 15), font, 0.5, (255,0,255), 1)
    
    face_number, xywh = faces.shape
    sub_img = {}

    
    for i in range(face_number):
        x, y, w, h = faces[i]
        sub_img[i] = gray[y:y+w,x:x+h] 
        cv2.imshow('face%s'%i,sub_img[i])      
        
    cv2.waitKey(0)
    
    k = cv2.waitKey(0) & 0xFF
    if k ==27:  #ESC
        cv2.destroyAllWindows()  
    elif k == ord('c'):
        for i in range(face_number):
            savedir = 'facesets/people' + str(i) + '.jpg'
            cv2.imwrite(savedir,sub_img[i])
        cv2.destroyAllWindows()

2.이미지 사이즈 바꾸기

In [17]:
def resizing(img,L=244):
    
    img2 = cv2.resize(img, (L,L), fx=1, fy=1, interpolation=cv2.INTER_AREA)
    
    return img2

3.detect face_landmark

In [3]:
#change from coordinate to np 
def shape_to_np(shape, dtype="int"):
    coords = np.zeros((68,2), dtype=dtype)
    
    #2-tuple of (x,y)-coordinates
    for i in range(68):
        coords[i] = (shape.part(i).x, shape.part(i).y)
        
    return coords

In [4]:
def landmark_detect(frame):
    
    #face detector(HOG-based), facial landmark predictor
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = resizing(gray,L=244)
    rects = detector(gray, 1)
    board = np.zeros((244,244,3), np.uint8)


    for (i,rect) in enumerate(rects):
    
#landmark shape 예측
        shape = predictor(gray, rect)
        shape = shape_to_np(shape)

        for (x, y) in shape:
            cv2.circle(board, (x, y), 1, (0, 0, 255), -1)

# show the output image with the face detections + facial landmarks
    cv2.imshow("Output", board)
    cv2.waitKey(0)
    
    k = cv2.waitKey(0) & 0xFF
    if k ==27:  #ESC
        cv2.destroyAllWindows()  
    elif k == ord('c'):
        cv2.imwrite('marksets/save.jpg',board)
        cv2.destroyAllWindows()

In [5]:
import cv2
import numpy as np
import dlib

imgfile = 'facesets/people2.jpg' #이미지 저장 디렉토리
frame = cv2.imread(imgfile, cv2.IMREAD_COLOR) # 이미지 불러오기

landmark_detect(frame)

4.face_emotion 분류하기(by microsoft face)

In [11]:
import requests
# If you are using a Jupyter notebook, uncomment the following line.
#%matplotlib inline
import matplotlib.pyplot as plt
from io import BytesIO

# Replace <Subscription Key> with your valid subscription key.
subscription_key = "173bd422f7a94a07887cd4a29b3c29fb"
assert subscription_key

# You must use the same region in your REST call as you used to get your
# subscription keys. For example, if you got your subscription keys from
# westus, replace "westcentralus" in the URI below with "westus".
#
# Free trial subscription keys are generated in the westcentralus region.
# If you use a free trial subscription key, you shouldn't need to change
# this region.
vision_base_url = "https://westcentralus.api.cognitive.microsoft.com/face/v1.0/detect"

# Set image_path to the local path of an image that you want to analyze.
image_path = "image/lena.jpg"

# Read the image into a byte array
image_data = open(image_path, "rb").read()
headers    = {'Ocp-Apim-Subscription-Key': subscription_key,
              'Content-Type': 'application/octet-stream'}
params = {
    'returnFaceId': 'false',
    'returnFaceLandmarks': 'false',
    'returnFaceAttributes': 'emotion'
}
response = requests.post(
    vision_base_url, headers=headers, params=params, data=image_data)
response.raise_for_status()

# The 'analysis' object contains various fields that describe the image. The most
# relevant caption for the image is obtained from the 'description' property.
analysis = response.json()
print(analysis)
emotion = analysis[0]['faceAttributes']['emotion']
print(emotion)

[{'faceRectangle': {'top': 887, 'left': 846, 'width': 612, 'height': 612}, 'faceAttributes': {'emotion': {'anger': 0.0, 'contempt': 0.0, 'disgust': 0.0, 'fear': 0.0, 'happiness': 0.0, 'neutral': 1.0, 'sadness': 0.0, 'surprise': 0.0}}}]
{'anger': 0.0, 'contempt': 0.0, 'disgust': 0.0, 'fear': 0.0, 'happiness': 0.0, 'neutral': 1.0, 'sadness': 0.0, 'surprise': 0.0}


In [15]:
import requests
# If you are using a Jupyter notebook, uncomment the following line.
#%matplotlib inline
import matplotlib.pyplot as plt
from io import BytesIO

#  ms_faceAPI를 사용하여 emotion dict로 추출
def face_classification(img_path):
    subscription_key = "173bd422f7a94a07887cd4a29b3c29fb"
    assert subscription_key

# You must use the same region in your REST call as you used to get your
# subscription keys. For example, if you got your subscription keys from
# westus, replace "westcentralus" in the URI below with "westus".
#
# Free trial subscription keys are generated in the westcentralus region.
# If you use a free trial subscription key, you shouldn't need to change
# this region.
    vision_base_url = "https://westcentralus.api.cognitive.microsoft.com/face/v1.0/detect"

# Set image_path to the local path of an image that you want to analyze.
#     image_path = "image/lena.jpg"

# Read the image into a byte array
    image_data = open(image_path, "rb").read()
    headers    = {'Ocp-Apim-Subscription-Key': subscription_key,
              'Content-Type': 'application/octet-stream'}
    params = {
    'returnFaceId': 'false',
    'returnFaceLandmarks': 'false',
    'returnFaceAttributes': 'emotion'
}
    response = requests.post(
    vision_base_url, headers=headers, params=params, data=image_data)
    response.raise_for_status()

# The 'analysis' object contains various fields that describe the image. The most
# relevant caption for the image is obtained from the 'description' property.
    analysis = response.json()
    print(analysis)
    emotion = analysis[0]['faceAttributes']['emotion']
    return emotion
    

5.face_emotion저장하기

In [16]:
import numpy as np
import cv2

def save_emotion(image_path,emotion):
    frame = cv2.imread(image_path, cv2.IMREAD_COLOR)
    
    #data 정제(score, class)
    emotion_list = [(k,emotion[k]) for k in emotion]
    emotion_score = [emotion[k] for k in emotion]
    emotion_class = [k for k in emotion]

    emotion_score = np.array(emotion_score)
    selected_class = emotion_class[np.argmax(emotion_score)]

    cv2.imwrite('image/emotion/'+selected_class+'/name.jpg',frame)
    

6.landmark 를 np data로 

In [15]:
import numpy as np
import cv2
import pickle

# numpy 한개당 (2,64) shape 으로 데이터 변환
def landmark_to_np(landmark_data):
    
    coords = np.zeros((68,2), dtype='float32')
    
    landmark = landmark_data.split(';')
    coords = np.array(landmark).reshape(68,2)
    return coords

6.landmark image으로 만들기

In [5]:
import numpy as np
import cv2
import pickle

def landmark_to_image():
    with open('faceset.pickle','rb') as f:
        faceset = pickle.load(f)
    
    #limit 414799
    for i in range(100):
        board = np.zeros((int(faceset['width'][i]),int(faceset['width'][i]),3), np.uint8)
        shape = landmark_to_np(faceset['facelandmark'][i])
        
        for (x, y) in shape:
            cv2.circle(board, (round(float(x)), round(float(y))), round(int(faceset['width'][i])*0.02), (0, 0, 255), -1)
        
        #그림크기 일정하게 통일하기
        resize_board = resizing(board,L=100)
        
        cv2.imwrite('marksets/'+str(i)+'.jpg',resize_board)
        

landmark_to_image()

In [5]:
#각 클래스별 데이터 갯수

import numpy as np
import cv2
import pickle

with open('faceset.pickle','rb') as f:
    faceset = pickle.load(f)
    
faceset['expression'][1]
for i in range(10):
    print(faceset['expression'].count(i))

74872
134409
25454
14089
6378
3803
24882
3750
33087
11644


0: Neutral, 1:Happiness, 2:Sadness, 3:Surprise, 4:Fear, 5:Disgust, 6:Anger, 7:Contempt, 8:None, 9:Uncertain, 10:No-Face

In [6]:
#np를 이용하여 각 클래스별 같은 양의 데이터 만들기
faceset_expression_np = np.array(faceset['expression'])

In [56]:
#trainset 만들기
index_0 = np.where(faceset_expression_np == 0)
index_1 = np.where(faceset_expression_np == 1)
index_2 = np.where(faceset_expression_np == 2)
index_3 = np.where(faceset_expression_np == 3)
index_4 = np.where(faceset_expression_np == 4)
index_5 = np.where(faceset_expression_np == 5)
index_6 = np.where(faceset_expression_np == 6)
index_7 = np.where(faceset_expression_np == 7)
print(np.shape(index_0))

train_index_0 = index_0[0][0:3000:1].tolist()
train_index_1 = index_1[0][0:3000:1].tolist()
train_index_2 = index_2[0][0:3000:1].tolist()
train_index_3 = index_3[0][0:3000:1].tolist()
train_index_4 = index_4[0][0:3000:1].tolist()
train_index_5 = index_5[0][0:3000:1].tolist()
train_index_6 = index_6[0][0:3000:1].tolist()
train_index_7 = index_7[0][0:3000:1].tolist()

train_index = []

train_index.append(train_index_0)
train_index.append(train_index_1)
train_index.append(train_index_2)
train_index.append(train_index_3)
train_index.append(train_index_4)
train_index.append(train_index_5)
train_index.append(train_index_6)
train_index.append(train_index_7)

train_index = np.array(train_index)
train_index = np.reshape(train_index, (-1))
np.shape(train_index)
train_index = train_index.tolist()

import random
random.shuffle(train_index)

#trainset 만들기
import csv

trainset = {}
trainset['width'] = []
trainset['facelandmark'] = []
trainset['expression'] = []

f = open('D:\workspace/training_AffectNet.csv','r')
frd = csv.reader(f)
facedata = []

for i,line in enumerate(frd):
    facedata.append(line)

for k in train_index:
    trainset['width'].append(facedata[k+1][4])
    trainset['facelandmark'].append(facedata[k+1][5])
    trainset['expression'].append(int(facedata[k+1][6]))

f.close()

import pickle
with open('face_train_set.pickle','wb') as f:
    pickle.dump(trainset, f)


(1, 74872)


In [9]:
#testSet 만들기
index_0 = np.where(faceset_expression_np == 0)
index_1 = np.where(faceset_expression_np == 1)
index_2 = np.where(faceset_expression_np == 2)
index_3 = np.where(faceset_expression_np == 3)
index_4 = np.where(faceset_expression_np == 4)
index_5 = np.where(faceset_expression_np == 5)
index_6 = np.where(faceset_expression_np == 6)
index_7 = np.where(faceset_expression_np == 7)
print(np.shape(index_0))

test_index_0 = index_0[0][3000:3500:1].tolist()
test_index_1 = index_1[0][3000:3500:1].tolist()
test_index_2 = index_2[0][3000:3500:1].tolist()
test_index_3 = index_3[0][3000:3500:1].tolist()
test_index_4 = index_4[0][3000:3500:1].tolist()
test_index_5 = index_5[0][3000:3500:1].tolist()
test_index_6 = index_6[0][3000:3500:1].tolist()
test_index_7 = index_7[0][3000:3500:1].tolist()

test_index = []

test_index.append(test_index_0)
test_index.append(test_index_1)
test_index.append(test_index_2)
test_index.append(test_index_3)
test_index.append(test_index_4)
test_index.append(test_index_5)
test_index.append(test_index_6)
test_index.append(test_index_7)

test_index = np.array(test_index)
test_index = np.reshape(test_index, (-1))
np.shape(test_index)
test_index = test_index.tolist()

import random
random.shuffle(test_index)

#trainset 만들기
import csv

testset = {}
testset['width'] = []
testset['facelandmark'] = []
testset['expression'] = []

f = open('D:\workspace/training_AffectNet.csv','r')
frd = csv.reader(f)
facedata = []

for i,line in enumerate(frd):
    facedata.append(line)

for k in test_index:
    testset['width'].append(facedata[k+1][4])
    testset['facelandmark'].append(facedata[k+1][5])
    testset['expression'].append(int(facedata[k+1][6]))

f.close()

import pickle
with open('face_test_set.pickle','wb') as f:
    pickle.dump(testset, f)


(1, 74872)


### x_train, y_train

In [32]:
with open('D:\workspace/data/AffectNet/face_train_set.pickle','rb') as f:
    trainset = pickle.load(f)

y_train = trainset['expression']
print(y_train)

[3, 1, 4, 2, 6, 1, 2, 4, 7, 1, 2, 5, 5, 1, 1, 0, 0, 3, 3, 5, 2, 1, 5, 2, 3, 1, 5, 2, 3, 0, 1, 0, 7, 2, 3, 7, 3, 3, 4, 1, 7, 5, 1, 7, 7, 2, 3, 6, 0, 0, 1, 3, 7, 3, 6, 7, 7, 4, 4, 1, 4, 1, 6, 3, 2, 3, 1, 5, 2, 3, 4, 0, 7, 2, 4, 1, 5, 0, 7, 4, 5, 2, 2, 7, 1, 1, 4, 6, 1, 1, 2, 6, 3, 6, 4, 6, 3, 3, 6, 1, 7, 5, 1, 1, 6, 0, 5, 6, 3, 3, 4, 5, 1, 5, 4, 4, 4, 4, 4, 4, 0, 0, 6, 6, 1, 3, 4, 7, 2, 1, 4, 3, 4, 4, 2, 1, 4, 7, 4, 4, 7, 2, 1, 6, 3, 2, 1, 0, 6, 0, 7, 3, 5, 5, 6, 1, 3, 1, 3, 4, 2, 6, 6, 1, 0, 5, 3, 6, 7, 4, 2, 3, 1, 2, 2, 1, 2, 5, 7, 5, 0, 6, 3, 5, 7, 3, 4, 7, 0, 5, 1, 4, 6, 4, 5, 5, 0, 4, 2, 7, 3, 3, 1, 6, 2, 0, 7, 1, 7, 5, 5, 2, 1, 2, 7, 1, 1, 4, 5, 5, 5, 1, 1, 3, 2, 7, 2, 4, 7, 4, 7, 0, 2, 6, 7, 1, 0, 5, 5, 7, 0, 6, 7, 1, 5, 2, 7, 6, 5, 3, 0, 6, 3, 2, 5, 5, 2, 3, 4, 5, 4, 6, 2, 0, 5, 1, 3, 7, 6, 0, 2, 5, 2, 1, 6, 7, 1, 6, 6, 2, 3, 2, 7, 3, 5, 3, 0, 4, 2, 4, 1, 6, 1, 7, 3, 6, 7, 0, 6, 6, 7, 1, 6, 0, 7, 2, 2, 3, 4, 5, 5, 0, 3, 3, 1, 2, 2, 7, 6, 5, 7, 5, 2, 5, 0, 2, 1, 3, 0, 7, 3, 2, 4, 

In [31]:
import numpy as np
import cv2
import pickle

def make_x():
    with open('D:\workspace/data/AffectNet/face_test_set.pickle','rb') as f:
        faceset = pickle.load(f)
    
    x_test = []
    
    #limit 414799
    for i in range(3000):
        board = np.zeros((int(faceset['width'][i]),int(faceset['width'][i]),3), np.uint8)
        shape = landmark_to_np(faceset['facelandmark'][i])
        
        for (x, y) in shape:
            cv2.circle(board, (round(float(x)), round(float(y))), round(int(faceset['width'][i])*0.03), (0, 0, 255), -1)
        
        #그림크기 일정하게 통일하기
        resize_board = resizing(board,L=32)
        x_test.append(resize_board)
    
    
    with open('x_test.pickle','wb') as f:
        pickle.dump(x_test, f)
    
make_x()

In [34]:
def make_y():
    with open('D:\workspace/data/AffectNet/face_test_set.pickle','rb') as f:
        faceset = pickle.load(f)
        
    y_train = trainset['expression']
    with open('y_test.pickle','wb') as f:
        pickle.dump(y_train, f)

make_y()